# Pytorch Tutorial

**Pushed by Facebook and designed for Python GPU running**

In [5]:
import torch
import numpy as np

## 1. Create Array

In [6]:
x1 = torch.ones(5,3)
x2 = torch.zeros(5,3)
x3 = torch.eye(4)
x4 = torch.rand(5,3)
x5 = torch.randn(5,3)

#Concatenate x1 and x2 at axis 1
x6 = torch.cat((x1,x2),1)
#Stack x1 and x2 at axis 1
x7 = torch.stack((x1,x2),1)
#reshape
x8 = x1.view(-1)
x9 = x2.expand(5,3)

#Numpy relation
a1 = np.array([1,2,4])
x10 = torch.from_numpy(a1)
a2 = x10.numpy()
a3 = [1,2,4]
x11 = torch.Tensor(a3)

print(x1)
print(x2)
print(x3)
print(x4)
print(x5)
print(x6)
print(x7)
print(x8)
print(x9)
print(a1)
print(x10)
print(a2)
print(x11)

tensor([[ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.]])
tensor([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]])
tensor([[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]])
tensor([[ 0.6230,  0.9187,  0.2897],
        [ 0.9383,  0.3139,  0.5042],
        [ 0.6413,  0.7564,  0.2275],
        [ 0.8676,  0.6462,  0.8919],
        [ 0.4324,  0.8229,  0.6536]])
tensor([[-1.3840, -0.0320,  0.2032],
        [-0.4949, -0.8910,  1.6536],
        [-0.9650, -0.8319, -0.1394],
        [-1.4508,  0.0410,  0.6471],
        [ 0.8073,  1.3711, -1.1532]])
tensor([[ 1.,  1.,  1.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  0.,  0.,  0.]])
tensor([[[ 1.,  1.,  1.],
         [ 0.,  0.,  0.]],



In [7]:
y1 = x1+x2
print(y1)

tensor([[ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.]])


## 2. Variable

In [8]:
from torch.autograd import Variable
#Variable includes data(), grad(), creator(), backward()
#requires_grad: Update or not,  volatile: need outcome but don't update
a = Variable(x1, requires_grad=True)
b = Variable(x2, volatile=True)
c = a + b

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """


## 3. Computation

1. 建立Variable
2. 建立optimizer

Train
1. 呼叫optimizer的zero_grad歸零grads
2. 操作現有參數得到loss
3. 呼叫loss backward算出所有梯度
4. optimizer.step更新梯度


In [9]:
from torch.optim import SGD
x1 = torch.ones(5,3)
x2 = torch.ones(5,3)
a = Variable(x1, requires_grad=True)
b = Variable(x2, requires_grad=True)

optimizer = SGD([a,b], lr=0.1)

#run for loop
for _ in range(10):
    
    optimizer.zero_grad()
    
    loss = (a+b).sum()
    loss.backward()
    optimizer.step()
    print(loss)

tensor(30.)
tensor(27.)
tensor(24.0000)
tensor(21.0000)
tensor(18.0000)
tensor(15.0000)
tensor(12.0000)
tensor(9.0000)
tensor(6.0000)
tensor(3.0000)


## 4. Neural Network 

1. 先用class建立model
2. 建立optimizer

train
1. 呼叫optimizer的zero_grad歸零grads
2. 操作現有參數得到loss
3. 呼叫loss backward算出所有梯度
4. optimizer.step更新梯度


In [10]:
import torch.nn as nn
import torch.nn.functional as F

In [14]:
#Define class for model
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  #Input channel, Output channel, 5x5 kernel size
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120) #Input, Output
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2)) # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # If the size is a square you can only specify a single number
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Model()
print(model.parameters())

<generator object Module.parameters at 0x10d526f68>


In [15]:
import torchvision
import torchvision.transforms as transforms


# torchvision数据集的输出是在[0, 1]范围内的PILImage图片。
# 我们此处使用归一化的方法将其转化为Tensor，数据范围为[-1, 1]

transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                             ])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, 
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, 
                                          shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr = 0.001)

for epoch in range(2):
    running_loss = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, lables = Variable(inputs), Variable(labels)
        
        optimizer.zero_grad()
        
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        if i % 2000 == 1999: # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[1,  2000] loss: 2.304
[1,  4000] loss: 2.301
[1,  6000] loss: 2.298
[1,  8000] loss: 2.291
[1, 10000] loss: 2.272
[1, 12000] loss: 2.192
[2,  2000] loss: 2.082
[2,  4000] loss: 2.014
[2,  6000] loss: 1.955
[2,  8000] loss: 1.922
[2, 10000] loss: 1.889
[2, 12000] loss: 1.854
Finished Training
